In [1]:
#Import dependencies
from pathlib import Path
import pandas as pd
import scipy.stats as stats

In [2]:
#Open filtered CSV file
traffic_injuries = Path("../Resources/road-traffic-injuries-2002-2010.csv")
df = pd.read_csv(traffic_injuries)

# Read in the data. 
df

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/road-traffic-injuries-2002-2010.csv'

In [34]:
#clean data to remove null values and columns 
clean_df = df.drop(columns = ["ind_definition","version"])
clean_df = clean_df.drop(clean_df[clean_df['ind_id'] == "END OF TABLE"].index)
clean_df = clean_df.drop(columns = ["ind_id", 'LL95CI_avmtrate', 'UL95CI_avmtrate',
       'avmtrate_se', 'avmtrate_rse', 'CA_decile_avmt', 'CA_RR_avmtrate',
       'groupquarters','LL95CI_poprate', 'UL95CI_poprate',
       'poprate_se', 'poprate_rse', 'CA_decile_pop', 'CA_RR_poprate',
       'avmttotal', 'avmtrate'])
clean_df = clean_df[clean_df['reportyear'] != '2002-2004']
clean_df = clean_df[clean_df['reportyear'] != '2005-2007']
clean_df = clean_df[clean_df['reportyear'] != '2008-2010']
clean_df = clean_df[clean_df['reportyear'] != '2006-2010']
clean_df = clean_df[clean_df['reportyear'] != '2002-2004']

clean_df

,reportyear,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,mode,severity,injuries,totalpop,poprate
0,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,All modes,Killed,193.0,2914663.0,6.62
1,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,All modes,Severe Injury,721.0,2914663.0,24.74
2,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,Bicyclist,Killed,7.0,2914663.0,0.24
3,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,Bicyclist,Severe Injury,49.0,2914663.0,1.68
4,2002,CO,6059.0,Orange,Orange,6059.0,Southern California,14.0,Bus,Severe Injury,1.0,2914663.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448944,2010,PL,56518.0,Penn Valley CDP,Nevada,6057.0,Northeast Sierra,6.0,Vehicles,Severe Injury,1.0,NaN,NaN
448945,2010,PL,80588.0,Truckee town,Nevada,6057.0,Northeast Sierra,6.0,All modes,Severe Injury,2.0,16180.0,12.36
448946,2010,PL,80588.0,Truckee town,Nevada,6057.0,Northeast Sierra,6.0,Car/Pickup,Severe Injury,1.0,16180.0,6.18
448947,2010,PL,80588.0,Truckee town,Nevada,6057.0,Northeast Sierra,6.0,Pedestrian,Severe Injury,1.0,16180.0,6.18


In [35]:
clean_df.isnull().sum()

reportyear           0
geotype              0
geotypevalue         0
geoname              0
county_name       2428
county_fips       2428
region_name     189601
region_code     189601
mode                 0
severity             0
injuries           469
totalpop        207793
poprate         208366
dtype: int64

In [36]:
#make modes of transport float with 1.0 representing whether all modes, bus, bike, etc. for each row
mode_float = pd.get_dummies(clean_df["mode"], dtype=float)
mode_float = mode_float.drop(columns = ["ND"])
mode_float

,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
448944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
448945,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
448946,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
448947,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
#make float for type of severity of accident. Killed or injured
severity_float = pd.get_dummies(clean_df["severity"], dtype=float)
#drop ND column added
severity_float = severity_float.drop(columns = ["ND"])
#make float list to concatenate since you cannot merge
floats = [mode_float, severity_float]
severity_float

,Killed,Severe Injury
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
448944,0.0,1.0
448945,0.0,1.0
448946,0.0,1.0
448947,0.0,1.0


In [38]:
#concatentate outer to get new df with 1.0 value denoting mode and injury
merged_mode_injury =  pd.concat(floats, axis=1, join="outer")
merged_mode_injury.head()

,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles,Killed,Severe Injury
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [39]:
#Create new dataframe, this is better than clean to be referenced when making graphs of data
new_df = pd.DataFrame(
    {"reportyear" : clean_df["reportyear"],
    "county_name" : clean_df["county_name"],
    "region_name" : clean_df["region_name"],
    "region_code" : clean_df["region_code"],
    "All modes" : merged_mode_injury["All modes"],
    "Bicyclist" : merged_mode_injury["Bicyclist"],
    "Bus": merged_mode_injury["Bus"],
    "Car/Pickup" : merged_mode_injury["Car/Pickup"],
    "Motorcycle" : merged_mode_injury["Motorcycle"],
    "Pedestrian" : merged_mode_injury["Pedestrian"],
    "Truck" : merged_mode_injury["Truck"],
    "Vehicles" : merged_mode_injury["Vehicles"],
    "Killed" : merged_mode_injury["Killed"],
    "Severe Injury" : merged_mode_injury["Severe Injury"],
    "injuries" : clean_df["injuries"],
    "totalpop" : clean_df["totalpop"],
    "poprate" : clean_df["poprate"]
 }
 )

new_df

,reportyear,county_name,region_name,region_code,All modes,Bicyclist,Bus,Car/Pickup,Motorcycle,Pedestrian,Truck,Vehicles,Killed,Severe Injury,injuries,totalpop,poprate
0,2002,Orange,Southern California,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,193.0,2914663.0,6.62
1,2002,Orange,Southern California,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,721.0,2914663.0,24.74
2,2002,Orange,Southern California,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,2914663.0,0.24
3,2002,Orange,Southern California,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0,2914663.0,1.68
4,2002,Orange,Southern California,14.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2914663.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448944,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN
448945,2010,Nevada,Northeast Sierra,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,16180.0,12.36
448946,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,16180.0,6.18
448947,2010,Nevada,Northeast Sierra,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,16180.0,6.18


In [45]:
#separate the new_df by years
collisions_2002 = new_df[new_df["reportyear"] == "2002"]
collisions_2003 = new_df[new_df["reportyear"] == "2003"]
collisions_2004 = new_df[new_df["reportyear"] == "2004"]
collisions_2005 = new_df[new_df["reportyear"] == "2005"]
collisions_2006 = new_df[new_df["reportyear"] == "2006"]
collisions_2007 = new_df[new_df["reportyear"] == "2007"]
collisions_2008 = new_df[new_df["reportyear"] == "2008"]
collisions_2009 = new_df[new_df["reportyear"] == "2009"]
collisions_2010 = new_df[new_df["reportyear"] == "2010"]

In [54]:
#Total collisions by type per year starting with 2002

#specify columns to total
specific_columns = ["All modes", "Bicyclist", "Bus", "Car/Pickup", "Motorcycle", "Pedestrian", "Truck", "Vehicles"]

totals_2002 = collisions_2002[specific_columns].sum()
totals_2002 = pd.DataFrame({"2002 Total":totals_2002})

totals_2003 = collisions_2003[specific_columns].sum()
totals_2003 = pd.DataFrame({"2003 Total":totals_2003})

totals_2004 = collisions_2004[specific_columns].sum()
totals_2004 = pd.DataFrame({"2004 Total":totals_2004})

totals_2005 = collisions_2005[specific_columns].sum()
totals_2005 = pd.DataFrame({"2005 Total":totals_2005})

totals_2006 = collisions_2006[specific_columns].sum()
totals_2006 = pd.DataFrame({"2006 Total":totals_2006})

totals_2007 = collisions_2007[specific_columns].sum()
totals_2007 = pd.DataFrame({"2007 Total":totals_2007})

totals_2008 = collisions_2008[specific_columns].sum()
totals_2008 = pd.DataFrame({"2008 Total":totals_2008})

totals_2009 = collisions_2009[specific_columns].sum()
totals_2009 = pd.DataFrame({"2009 Total":totals_2009})

totals_2010 = collisions_2010[specific_columns].sum()
totals_2010 = pd.DataFrame({"2010 Total":totals_2010})

all_totals = new_df[specific_columns].sum()
all_totals = pd.DataFrame({"Overall Total": all_totals})

#Combine totals for each year into a single dataframe
yearly_totals_mode = pd.merge(totals_2002, totals_2003, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2004, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2005, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2006, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2007, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2008, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2009, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, totals_2010, left_index=True, right_index=True)
yearly_totals_mode = pd.merge(yearly_totals_mode, all_totals, left_index=True, right_index=True)

yearly_totals_mode



,2002 Total,2003 Total,2004 Total,2005 Total,2006 Total,2007 Total,2008 Total,2009 Total,2010 Total,Overall Total
All modes,8550.0,8498.0,8503.0,8456.0,8502.0,8382.0,7862.0,7481.0,7259.0,73493.0
Bicyclist,909.0,1050.0,1225.0,1209.0,1283.0,1179.0,1270.0,1262.0,1273.0,10660.0
Bus,71.0,57.0,45.0,81.0,79.0,70.0,76.0,64.0,65.0,608.0
Car/Pickup,6857.0,6518.0,6495.0,6356.0,6291.0,6025.0,5413.0,5061.0,4722.0,53738.0
Motorcycle,1960.0,2087.0,2067.0,2115.0,2261.0,2477.0,2500.0,2217.0,2183.0,19867.0
Pedestrian,2069.0,2434.0,2622.0,2718.0,2679.0,2683.0,2556.0,2506.0,2474.0,22741.0
Truck,395.0,353.0,359.0,360.0,340.0,356.0,314.0,269.0,244.0,2990.0
Vehicles,7494.0,7229.0,7144.0,7064.0,7096.0,6939.0,6427.0,5984.0,5678.0,61055.0


In [56]:
#total killed vs severe injury per year
death_injury = ["Killed", "Severe Injury"]

di_totals_2002 = collisions_2002[death_injury].sum()
di_totals_2002 = pd.DataFrame({"2002 Total":di_totals_2002})

di_totals_2003 = collisions_2003[death_injury].sum()
di_totals_2003 = pd.DataFrame({"2003 Total":di_totals_2003})

di_totals_2004 = collisions_2004[death_injury].sum()
di_totals_2004 = pd.DataFrame({"2004 Total":di_totals_2004})

di_totals_2005 = collisions_2005[death_injury].sum()
di_totals_2005 = pd.DataFrame({"2005 Total":di_totals_2005})

di_totals_2006 = collisions_2006[death_injury].sum()
di_totals_2006 = pd.DataFrame({"2006 Total":di_totals_2006})

di_totals_2007 = collisions_2007[death_injury].sum()
di_totals_2007 = pd.DataFrame({"2007 Total":di_totals_2007})

di_totals_2008 = collisions_2008[death_injury].sum()
di_totals_2008 = pd.DataFrame({"2008 Total":di_totals_2008})

di_totals_2009 = collisions_2009[death_injury].sum()
di_totals_2009 = pd.DataFrame({"2009 Total":di_totals_2009})

di_totals_2010 = collisions_2010[death_injury].sum()
di_totals_2010 = pd.DataFrame({"2010 Total":di_totals_2010})

di_all_totals = new_df[death_injury].sum()
di_all_totals = pd.DataFrame({"Overall Total": di_all_totals})

#Combine totals for each year into a single dataframe
yearly_totals_di = pd.merge(di_totals_2002, di_totals_2003, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2004, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2005, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2006, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2007, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2008, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2009, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_totals_2010, left_index=True, right_index=True)
yearly_totals_di = pd.merge(yearly_totals_di, di_all_totals, left_index=True, right_index=True)

yearly_totals_di

,2002 Total,2003 Total,2004 Total,2005 Total,2006 Total,2007 Total,2008 Total,2009 Total,2010 Total,Overall Total
Killed,8715.0,8933.0,8762.0,8925.0,9063.0,8586.0,7835.0,7324.0,6869.0,75012.0
Severe Injury,19590.0,19293.0,19698.0,19434.0,19468.0,19525.0,18583.0,17520.0,17029.0,170140.0


In [ ]:
#Make graph comparing every mode of travel by every type of injury
#make graph breaking down how many were killed or severly injured by type of travel, maybe calculate it percent to make pie graph or do a bar chart
#compare total accidents per region
#graphs comparing accidnets and miles traveled? Or region and miles traveled? Might help with linear regression later. See comment in box with correlation matrix to see how to use miles traveled
#Do these for every year
#Do these summing up all the years, maybe make dataframes specific to a year
#any other ideas?

In [83]:
#need to do linear regression model of graphs. What does the data tell us? is there trend in injury over year? by region? By mode of transport?
#What about by miles traveled?(this will be found all the way back in beginnning under "df["avmtrate"] look at the excel dictionary I think we can still use this. If used be sure to remove rows are same as clean_df or new_df) 


# Assuming df is your DataFrame
correlation_matrix = clean_df[["geotypevalue","county_fips", "region_code", "injuries", "totalpop", "poprate"]].describe()

#I dont think this is the correlation we are looking for in regard to final project presentation. I think we should make a dataframe that
#is summed up and more about accident per region, modes of transport, and miles traveled.

# Display the correlation matrix
print(correlation_matrix)
#clean_df.info()

       geotypevalue    county_fips   region_code       injuries      totalpop  \
count  4.489480e+05  445371.000000  94066.000000  447565.000000  1.133240e+05   
mean   4.816763e+09    6055.795018      8.762082       6.797583  2.552469e+05   
std    3.169910e+09      27.275320      4.892614     122.888407  1.868938e+06   
min    1.000000e+00    6001.000000      1.000000       0.200000  0.000000e+00   
25%    6.011000e+09    6037.000000      4.000000       0.670000  4.424000e+03   
50%    6.037573e+09    6059.000000     10.000000       1.000000  9.174000e+03   
75%    6.071009e+09    6073.000000     14.000000       2.000000  6.256300e+04   
max    1.000000e+11    6115.000000     14.000000   13578.000000  3.725396e+07   

             poprate  
count  111634.000000  
mean       27.683685  
std       244.278075  
min         0.000000  
25%         3.790000  
50%         7.930000  
75%        19.090000  
max     26000.000000  
